# Group Project / Assignment 4: Instruction finetuning a Llama-3.2 model
**Assignment due 21 April 11:59pm**

Welcome to the fourth and final assignment for 50.055 Machine Learning Operations. The third and fourth assignment together form the course group project. You will continue the work on a chatbot which can answer questions about SUTD to prospective students.


**This assignment is a group assignment.**

- Read the instructions in this notebook carefully
- Add your solution code and answers in the appropriate places. The questions are marked as **QUESTION:**, the places where you need to add your code and text answers are marked as **ADD YOUR SOLUTION HERE**. The assignment is more open-ended than previous assignments, i.e. you have more freedom how to solve the problem and how to structure your code.
- The completed notebook, including your added code and generated output will be your submission for the assignment.
- The notebook should execute without errors from start to finish when you select "Restart Kernel and Run All Cells..". Please test this before submission.
- Use the SUTD Education Cluster to solve and test the assignment. If you work on another environment, minimally test your work on the SUTD Education Cluster.

**Rubric for assessment**

Your submission will be graded using the following criteria.
1. Code executes: your code should execute without errors. The SUTD Education cluster should be used to ensure the same execution environment.
2. Correctness: the code should produce the correct result or the text answer should state the factual correct answer.
3. Style: your code should be written in a way that is clean and efficient. Your text answers should be relevant, concise and easy to understand.
4. Partial marks will be awarded for partially correct solutions.
5. Creativity and innovation: in this assignment you have more freedom to design your solution, compared to the first assignments. You can show of your creativity and innovative mindset.
6. There is a maximum of 310 points for this assignment.

**ChatGPT policy**

If you use AI tools, such as ChatGPT, to solve the assignment questions, you need to be transparent about its use and mark AI-generated content as such. In particular, you should include the following in addition to your final answer:
- A copy or screenshot of the prompt you used
- The name of the AI model
- The AI generated output
- An explanation why the answer is correct or what you had to change to arrive at the correct answer

**Assignment Notes:** Please make sure to save the notebook as you go along. Submission Instructions are located at the bottom of the notebook.



### Finetuning LLMs

The goal of the assignment is to build a more advanced chatbot that can talk to prospective students and answer questions about SUTD.

We will finetune a smaller 1B LLM on question-answer pairs which we synthetically generate. Then we will compare the finetuned and non-finetuned LLMs with and without RAG to see if we were able to improve the SUTD chatbot answer quality.

We'll be leveraging `langchain`, `llama 3.2` and `Google AI STudio with Gemini 2.0`.

Check out the docs:
- [LangChain](https://docs.langchain.com/docs/)
- [Llama 3.2](https://www.llama.com/docs/model-cards-and-prompt-formats/llama3_2/)
- [Google AI Studio](https://aistudio.google.com/)

Note: Google AI Studio provides a lot of free tokens but has certain rate limits. Write your code in a way that it can handle these limits.

# Install dependencies
Use pip to install all required dependencies of this assignment in the cell below. Make sure to test this on the SUTD cluster as different environments have different software pre-installed.  

In [1]:
# QUESTION: Install and import all required packages
# The rest of your code should execute without any import or dependency errors.

# **--- ADD YOUR SOLUTION HERE (10 points) ---**
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers
!pip install datasets
!pip install accelerate
!pip install huggingface_hub
!pip install langchain langchain-core langchain-community langchain-experimental langchain-google-genai langchain_openai
!pip install google-generativeai
!pip install flashrank
!pip install openai
!pip install sentence-transformers
!pip install dotenv
!pip install pydantic
!pip install unsloth
!pip install accelerate
!pip install peft
!pip uninstall trl -y && pip install --no-cache-dir --force-reinstall --no-deps "trl<0.15.0"
!pip install bitsandbytes
!pip install faiss-cpu
!pip install gradio

Looking in indexes: https://download.pytorch.org/whl/cu118
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 85.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 130.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 16.8 MB/s e

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.3 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.17
    Uninstalling google-ai-generativelanguage-0.6.17:
      Successfully uninstalled google-ai-generativelanguage-0.6.17
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.3 requires google-ai-generativelanguage<0.7.0,>=0.6.16, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.8/127.8 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 121.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found 

Found existing installation: trl 0.15.2
Uninstalling trl-0.15.2:
  Successfully uninstalled trl-0.15.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 85.1 MB/s eta 0:00:00


In [21]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 133.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 8.4 MB/s eta 0:00:00


# Importing libraries

In [ ]:
# finetuning imports
import unsloth
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

# TODO: check the difference between ChatGoogleGenerativeAI and GoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from langchain_core.pydantic_v1 import BaseModel, Field
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from datasets import load_dataset
from huggingface_hub import login
from transformers import pipeline, TrainingArguments
from trl import SFTTrainer
from unsloth import FastLanguageModel
from unsloth import is_bfloat16_supported
from unsloth import to_sharegpt, standardize_sharegpt
from peft import PeftModel
from typing import List
import os
from pydantic import BaseModel
import time
import json
import torch, gc
import pandas as pd
import matplotlib.pyplot as plt
import re

# rag imports
from openai import OpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.vectorstores import FAISS
from flashrank import Ranker, RerankRequest
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from langchain.chains import RetrievalQA
from sentence_transformers import SentenceTransformer
import gradio as gr

load_dotenv()
GOOGLE_GENAI_API_KEY = os.getenv("GOOGLE_GENAI_API_KEY")
HUGGINGFACE_TOKEN = os.getenv("HUGGINGFACE_TOKEN")
USERNAME = os.getenv("HUGGINGFACE_USERNAME")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# rag imports
MARKDOWN_PATH = "data/markdown/markdown_data.json"
HTML_PATH = "data/html/html_data.json"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
EMBEDDING_MODEL = "text-embedding-ada-002"
TOP_K = 5
OUTPUT_DIR = "vector_store"
HUGGINGFACE_TOKEN = os.getenv("HUGGINGFACE_TOKEN")
MODEL_DIR = "models"
JSON_PATH = "data/data.json"
client = OpenAI(api_key=OPENAI_API_KEY)

login(token=HUGGINGFACE_TOKEN)

# Generate training data
The first step of the assignment is generating synthetic question-answer pairs which can be used for finetuning an LLM model.
Use the Google AI studio with the Gemini models to create -high-quality QA training data.


In [ ]:
# QUESTION: Use langchain and the Google AI Studio APIs and a model from the Gemini 2.0 family
# to create a text-generation chain that can produce and parse JSON output.
# Test it by having the LLM generate a JSON array of 3 fruits

#--- ADD YOUR SOLUTION HERE (20 points)---

# --------------------------------------------------------------------------------

# we are creating synthetic data that contains only questions and responses but not on multiple tasks, because this is a chatbot application
model = ChatGoogleGenerativeAI(google_api_key=GOOGLE_GENAI_API_KEY, model="gemini-2.0-flash", temperature=0.2, convert_system_message_to_human=True)
parser = JsonOutputParser()
input_prompt = "Generate a JSON array containing exactly 3 fruit names."
prompt = ChatPromptTemplate.from_messages([
    ("system", """
    You are a helpful assistant that generates structured JSON data.
    Your response should ONLY contain valid JSON without any additional text.
    Do not include explanation, notes, or markdown formatting.
    """),
    ("human", "{input_text}")
])
chain = ( prompt | model | parser )
response = chain.invoke({"input_text": input_prompt})
print(response)

# --------------------------------------------------------------------------------

# imo there is a better way to do this which is using pydantic and langchain
class FruitList(BaseModel):
    fruits: List[str] = Field(description="A list of fruit objects", min_length=3, max_length=3)

pydantic_parser = JsonOutputParser(pydantic_object=FruitList)
prompt = PromptTemplate(
    template  = "Answer the user's question.\n\n{format_instructions}\n\n{input}",
    input_variables = ["input"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = ( prompt | model | pydantic_parser )
response = chain.invoke({"input": input_prompt})
print(response)

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:388: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


['apple', 'banana', 'orange']


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:388: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


['apple', 'banana', 'orange']


## Generate topics
When generating data, it is often helpful to guide the generation process through some hierachical structure.
Before we create question-answer pairs, let's generate some topics which the questions should be about.



In [ ]:
# QUESTION: Create a function 'generate_topics' which generates topics which prospective students might care about.
#
# Generate a list of 20 topics

#--- ADD YOUR SOLUTION HERE (20 points)---
def generate_topics(num = 20):
    class TopicsList(BaseModel):
        topics: List[str] = Field(description=f"A list of {num} topics that prospective students might care about", min_length=num, max_length=num)

    # if the file already exists don't run instead return the topics from the file
    output_dir = "data"

    if os.path.exists(output_dir) == False:
        os.makedirs(output_dir)

    filename = f"data/topics.json"

    model = ChatGoogleGenerativeAI(
        google_api_key=GOOGLE_GENAI_API_KEY,
        model="gemini-2.0-flash",
        temperature=0.8, # using this value for more diverse topics
        convert_system_message_to_human=True #the system message is reformatted or “converted” into a style that resembles the way a human would pose a question or comment
    )

    pydantic_parser = JsonOutputParser(pydantic_object=TopicsList)
    pydantic_parser = JsonOutputParser(pydantic_object=TopicsList)

    prompt = PromptTemplate(
        template=(
            "List out topics a prospective student might be interested in when "
            "Think of real concerns such as academic quality, campus life, tuition, "
            "social environment, and career opportunities.\n\n"
            "{format_instructions}\n\n"
            "{input}"
        ),
        input_variables=["input"],
        partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
    )
    input_prompt = (
        f"Please list exactly {num} topics that capture what prospective students care about "
        "when choosing a university like SUTD."
    )

    chain = prompt | model | parser
    response = chain.invoke({"input": input_prompt})

    topics = response["topics"]

    # save to disk otherwise
    with open(filename, 'w') as file:
        json.dump(topics, file)

    return topics

In [ ]:
# test topic generation
print(generate_topics(3))

/usr/local/lib/python3.11/dist-packages/pydantic/json_schema.py:2324: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='A list of 3 topics that prospective students might care about' min_length=3 max_length=3 extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:388: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


['Academic Rigor and Curriculum', 'Career Prospects and Industry Connections', 'Campus Culture and Student Life']


In [ ]:
# Generate a list of 20 topics
# We save a copy to disk and reload it from there if the file exists

# the function to generate topics and saves it to disk as topics_20.json
print(generate_topics(20))

c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\pydantic\json_schema.py:2324: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='A list of 20 topics that prospective students might care about' min_length=20 max_length=20 extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:367: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


['Academic Reputation and Ranking', 'Specific Program Strengths (e.g., Engineering, Design)', 'Faculty Expertise and Research Opportunities', 'Hands-on Learning and Project-Based Curriculum', 'Internship Opportunities and Industry Connections', 'Career Services and Graduate Employment Rate', 'Tuition Fees and Financial Aid Options', 'Scholarship Availability and Eligibility Criteria', 'Cost of Living in Singapore', 'Accommodation Options (on-campus and off-campus)', 'Campus Facilities and Resources (labs, library, maker spaces)', 'Student-Faculty Ratio and Class Sizes', 'Student Support Services (academic advising, counseling)', 'Campus Culture and Student Life', 'Diversity and Inclusion Initiatives', 'Extracurricular Activities and Clubs', 'Location and Accessibility of the University', 'Opportunities for International Exchange Programs', 'Networking Opportunities with Alumni', 'Safety and Security on Campus']


## Generate questions
Now generate a set of questions about each topic

In [ ]:
# QUESTION: Create a function 'generate_questions' which generates quetions about a given topic.
# Generate a list of 10 questions per topics. In total you should have 200 questions.
#

#--- ADD YOUR SOLUTION HERE (20 points)---
# TODO: rememeber to add timeout to the function
def generate_questions(topic, num=10):
    output_dir = "data"

    if os.path.exists(output_dir) == False:
        os.makedirs(output_dir)

    model = ChatGoogleGenerativeAI(
        google_api_key=GOOGLE_GENAI_API_KEY,
        model="gemini-2.0-flash",
        temperature=0.4, # hyperparameter can be changed
        convert_system_message_to_human=True
    )

    class Questions(BaseModel):
        questions: List[str] = Field(
            description=f"A list of {num} questions about a specific topic",
            min_length=num,
            max_length=num
        )

    parser = JsonOutputParser(pydantic_object=Questions)

    # i think the best thing is to avoid specifics to the university to avoid confusion
    prompt = PromptTemplate(
        template=(
            "Imagine you are a prospective university student wanting to know more about a specific aspect. "
            "For the topic provided, generate exactly {num_questions} questions that you might naturally ask. "
            "Topic: {topic}\n\n"
            "{format_instructions}"
        ),
        input_variables=["topic", "num_questions"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )
    chain = prompt | model | parser

    questions_data = {}

    try:
        print(f"Generating questions for topic: {topic}")
        response = chain.invoke({"topic": topic, "num_questions": num})
        questions_data[topic] = response["questions"]

    except Exception as e:
        print(f"Error generating questions for topic '{topic}': {e}")

    return questions_data

In [ ]:
# test it
print(generate_questions("Academic Reputation and Program Quality", 3))

c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\pydantic\json_schema.py:2324: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='A list of 3 questions about a specific topic' min_length=3 max_length=3 extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:367: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Generating questions for topic: Academic Reputation and Program Quality
{'Academic Reputation and Program Quality': ['What specific accreditations does the [Program Name] program hold, and how do these accreditations benefit students?', 'Could you provide data on graduate employment rates and the types of positions graduates typically obtain after completing the [Program Name] program?', 'How does the university ensure the curriculum for the [Program Name] program remains current and relevant to industry trends and advancements?']}


In [ ]:
# # QUESTION: Now let's put it together and generate 10 questions for each topic. Save the questions in a local file.

#--- ADD YOUR SOLUTION HERE (20 points)---
with open("data/topics.json", "r") as f:
    topics = json.load(f)

rate_limit = 15
requests_made = 0
all_questions = {}

for i, topic in enumerate(topics):
    if requests_made >= rate_limit:
        time.sleep(60)
        requests_made = 0
    response = generate_questions(topic)
    all_questions[topic] = response[topic]
    requests_made += 1

with open("data/questions.json", "w") as f:
    json.dump(all_questions, f, indent=2)

c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\pydantic\json_schema.py:2324: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='A list of 10 questions about a specific topic' min_length=10 max_length=10 extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:367: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Generating questions for topic: Academic Reputation and Ranking


c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\pydantic\json_schema.py:2324: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='A list of 10 questions about a specific topic' min_length=10 max_length=10 extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:367: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Generating questions for topic: Specific Program Strengths (e.g., Engineering, Design)


c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\pydantic\json_schema.py:2324: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='A list of 10 questions about a specific topic' min_length=10 max_length=10 extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:367: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


...

Generating questions for topic: Opportunities for International Exchange Programs


c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\pydantic\json_schema.py:2324: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='A list of 10 questions about a specific topic' min_length=10 max_length=10 extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:367: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Generating questions for topic: Networking Opportunities with Alumni


c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\pydantic\json_schema.py:2324: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='A list of 10 questions about a specific topic' min_length=10 max_length=10 extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:367: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Generating questions for topic: Safety and Security on Campus


## Generate Answers

Now create answers for the questions.

You can use the Google AI Studio Gemini model (assuming that they are good enough to generate good answers), your RAG system from assignment 3 or any other method you choose to generate answers for your question dataset.

Note: it is normal that some LLM calls fail, even with retry, so maybe you end up with less than 200 QA pairs but it should be at least 160 QA pairs.

In [ ]:
# QUESTION: Generate answers to al your questions using Gemini, your SUTD RAG system or any other method.
# Split your dataset in to 80% training and 20% test dataset.
# Store all questions and answer pairs in a huggingface dataset `sutd_qa_dataset` and push it to your Huggingface hub.

#--- ADD YOUR SOLUTION HERE (40 points)---

# generate answers to al the questions using Gemini and then split the dataset in to 80% training and 20% test dataset.
def generate_answer(question):
    class Answer(BaseModel):
        answer: str = Field(description="The answer to the question")

    parser = JsonOutputParser(pydantic_object=Answer)
    prompt = PromptTemplate(
        template=(
            "Answer the following question in a friendly, clear, and brief manner, as though you "
            "are advising a prospective student. Use simple language and get straight to the point.\n\n"
            "{format_instructions}\n\n"
            "{question}"
        ),
        input_variables=["question"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    model = ChatGoogleGenerativeAI(
        google_api_key=GOOGLE_GENAI_API_KEY,
        model="gemini-2.0-flash",
        temperature=0.3,
        convert_system_message_to_human=True
    )
    chain = prompt | model | parser
    return chain.invoke({"question": question})

In [ ]:
# now split the dataset in to 80% training and 20% test dataset

# load the questions from the json
with open("data/questions.json", "r") as f:
    questions_data = json.load(f)

# flattening the questions data with topic as the key
flat_questions = []
for topic, questions in questions_data.items():
    for question in questions:
        flat_questions.append({"topic": topic, "question": question})

questions_df = pd.DataFrame(flat_questions)

# splits
train_df, test_df = train_test_split(questions_df, test_size=0.2, random_state=42)

print(f"Training size: {len(train_df)}")
print(f"Test size: {len(test_df)}")

# save
train_df.to_csv("data/train.csv", index=False)
test_df.to_csv("data/test.csv", index=False)

Training size: 160
Test size: 40


In [ ]:
# test the chain
question = "When was SUTD founded?"

# Now run the answer generation chain
response = generate_answer(question)
print("\nModel Response:")
print(response["answer"])

c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\pydantic\json_schema.py:2324: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='The answer to the question' extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:367: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Model Response:
SUTD was founded in 2009.


In [ ]:
# now run the chain for all questions to collect context and generate answers

# load the questions
def process_dataset(csv_file, output_file):
    # check if the output file already exists to resume from previous run (IT CRASHED)
    try:
        df = pd.read_csv(output_file)
        print(f"Resuming from existing file: {output_file}")
    except FileNotFoundError:
        df = pd.read_csv(csv_file)
        df['answer'] = None  # add empty answer column to be filled in

    requests_made = 0
    rate_limit = 12  # reduced from 15 to 12 to be safer
    count = 0
    total = len(df)
    start_time = time.time()

    # track which rows we've already processed
    processed_rows = 0
    for index, row in df.iterrows():
        # skip rows that already have answers
        if pd.notna(row.get('answer')):
            processed_rows += 1
            continue

        question = row['question']

        # rate limit by time window (60 seconds) and requests made
        current_time = time.time()
        elapsed_time = current_time - start_time
        if requests_made >= rate_limit:
            remaining_time = 60 - elapsed_time
            # if requests made is greater than rate limit, and time has elapsed is greater than 60 seconds, sleep for remaining time
            if remaining_time > 0:
                print(f"Rate limit reached. Sleeping for {remaining_time:.2f} seconds...")
                time.sleep(remaining_time)
            # reset counter and time window
            requests_made = 0
            start_time = time.time()

        # generate answer
        try:
            answer = generate_answer(question=question)
            df.at[index, 'answer'] = answer["answer"]

            # Save progress after each successful answer
            if processed_rows % 5 == 0:  # Save every 5 processed items
                df.to_csv(output_file, index=False)

            count += 1
            requests_made += 1
            processed_rows += 1
            print(f"Processing {csv_file}: {processed_rows}/{total}")

        except Exception as e:
            print(f"Error processing question: {question}")
            print(f"Error: {e}")
            # save progress asap
            df.to_csv(output_file, index=False)

            # if quota exceeded, wait longer
            if "429" in str(e) or "exceeded" in str(e).lower() or "ResourceExhausted" in str(e):
                wait_time = 120  # 2min wait time
                print(f"API quota exceeded. Waiting for {wait_time} seconds before retrying...")
                time.sleep(wait_time)
                requests_made = 0  # reset
                start_time = time.time()
            continue

    # save the complete dataset
    df.to_csv(output_file, index=False)
    print(f"Finished writing {output_file}")
    return df

print("Generating answers for the training dataset...")
train_with_answers = process_dataset("data/train.csv", "data/train.csv")

print("Generating answers for the testing dataset...")
test_with_answers = process_dataset("data/test.csv", "data/test.csv")

Generating answers for the training dataset...
Resuming from existing file: data/train.csv


c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\pydantic\json_schema.py:2324: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='The answer to the question' extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:367: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Processing data/train.csv: 1/160


c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\pydantic\json_schema.py:2324: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='The answer to the question' extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:367: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Processing data/train.csv: 2/160


...

Processing data/test.csv: 38/40


c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\pydantic\json_schema.py:2324: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='The answer to the question' extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:367: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Processing data/test.csv: 39/40


c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\pydantic\json_schema.py:2324: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='The answer to the question' extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:367: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Processing data/test.csv: 40/40
Finished writing data/test.csv


In [ ]:
# push to huggingface
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

# convert to huggingface format
train_dataset = Dataset.from_pandas(train_with_answers)
test_dataset = Dataset.from_pandas(test_with_answers)

sutd_qa_dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

# huggingface push
# TODO: change username to .env
sutd_qa_dataset.push_to_hub(f"{USERNAME}/sutd_qa_dataset")

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.39s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/adi0308/sutd_qa_dataset/commit/0a7d8b193b7e016f64c87ec56af251f6e2f84340', commit_message='Upload dataset', commit_description='', oid='0a7d8b193b7e016f64c87ec56af251f6e2f84340', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/adi0308/sutd_qa_dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='adi0308/sutd_qa_dataset'), pr_revision=None, pr_num=None)

# Finetune Llama 3.2 1B model

Now use your SUTD QA dataset training data set to finetune a smaller Llama 3.2 1B LLM using parameter-efficient finetuning (PEFT).
We recommend the unsloth library but you are free to choose other frameworks. You can decide the parameters for the finetuning.
Push your finetuned model to Huggingface.

Then we will compare the finetuned and non-finetuned LLMs with and without RAG to see if we were able to improve the SUTD chatbot answer quality.


In [ ]:
# load the dataset from huggingface
raw = load_dataset(f"{USERNAME}/sutd_qa_dataset")

sg_train = to_sharegpt(
    raw["train"],
    merged_prompt = "Question: {question}\nAnswer:",
    output_column_name = "answer",
    conversation_extension = 0,
)
sg_test = to_sharegpt(
    raw["test"],
    merged_prompt = "Question: {question}\nAnswer:",
    output_column_name = "answer",
    conversation_extension = 0,
)

print("before standardize:", sg_train.column_names)
chat_train = standardize_sharegpt(sg_train)
chat_test  = standardize_sharegpt(sg_test)

print("after standardize:", chat_train.column_names)

before standardize: ['conversations']
after standardize: ['conversations']


In [ ]:
# QUESTION: Finetune a Llama 3.2 1B model on the training split of your SUTD QA dataset.
# You need to prepare your dataset accordingly and set the hyperparameters for the training.
# Push your finetuned model to the Hugginface model hub {YOUR_HF_NAME}/llama-3.2-1B-sutdqa

#--- ADD YOUR SOLUTION HERE (50 points)---
# following the docs at: https://docs.unsloth.ai/basics/tutorial-how-to-finetune-llama-3-and-use-in-ollama
# using base llama 3.2 and not instruct tuned: https://huggingface.co/unsloth/Llama-3.2-1B-bnb-4bit
def fmt_qa(batch):
    return [f"Question: {q}\nAnswer: {a}"
            for q,a in zip(batch["question"], batch["answer"])]

max_seq_length = 512
base_model, tokenizer = FastLanguageModel.from_pretrained(
    "unsloth/Llama-3.2-1B-bnb-4bit",
    load_in_4bit=True,
    device_map="auto",
    max_seq_length=max_seq_length,
)
tokenizer.pad_token_id = tokenizer.eos_token_id

model = FastLanguageModel.get_peft_model(
    base_model,
    r=16,
    target_modules=[
      "q_proj","k_proj","v_proj","o_proj",
      "gate_proj","up_proj","down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0.0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
)

def fmt_qa(batch):
    return [
        f"Question: {q}\nAnswer: {a}"
        for q, a in zip(batch["question"], batch["answer"])
    ]

OUTPUT_MODEL = f"{USERNAME}/llama-3.2-1B-sutdqa"
training_args = TrainingArguments(
    output_dir = "outputs",
    num_train_epochs = 5,
    per_device_train_batch_size= 2,
    gradient_accumulation_steps= 4,
    learning_rate = 1e-4,
    fp16 = not is_bfloat16_supported(),
    bf16 = is_bfloat16_supported(),
    save_strategy = "epoch",
    push_to_hub = True,
    hub_model_id = OUTPUT_MODEL,
    hub_token = HUGGINGFACE_TOKEN,
    logging_steps = 10,
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = raw["train"],
    eval_dataset = raw["test"],
    dataset_text_field = None,
    formatting_func = fmt_qa,
    max_seq_length= max_seq_length,
    packing = False,
    args = training_args,
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu118. CUDA: 8.9. CUDA Toolkit: 11.8. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
# finetuning run
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 160 | Num Epochs = 5 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 11,272,192/1,000,000,000 (1.13% trained)


Step,Training Loss
10,2.396300
20,1.884600
30,1.531000
40,1.474400
50,1.307400
60,1.256300
70,1.143200
80,1.117300
90,1.040300
100,1.034500


TrainOutput(global_step=100, training_loss=1.418526668548584, metrics={'train_runtime': 91.2855, 'train_samples_per_second': 8.764, 'train_steps_per_second': 1.095, 'total_flos': 262697930932224.0, 'train_loss': 1.418526668548584, 'epoch': 5.0})

In [ ]:
# push to huggingface
trainer.push_to_hub()

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/adi0308/llama-3.2-1B-sutdqa/commit/8b73454a5d3ae8fa2dc722db41e595f079d82c30', commit_message='End of training', commit_description='', oid='8b73454a5d3ae8fa2dc722db41e595f079d82c30', pr_url=None, repo_url=RepoUrl('https://huggingface.co/adi0308/llama-3.2-1B-sutdqa', endpoint='https://huggingface.co', repo_type='model', repo_id='adi0308/llama-3.2-1B-sutdqa'), pr_revision=None, pr_num=None)

In [5]:
# QUESTION: Load a non-finetuned Llama 3.2 1B model and your finetuned SUTD QA Llama 3.2 1B model
# Ask it a simple test question (e.g. "What is special about SUTD?") to check that both models can generated answers

#--- ADD YOUR SOLUTION HERE (10 points)---
# load the models using the huggingface pipeline from langchain
# 1) Base (un‑finetuned) Llama‑3.2 4‑bit
base_model, tokenizer = FastLanguageModel.from_pretrained(
    "unsloth/Llama-3.2-1B-bnb-4bit",
    load_in_4bit=True,
    device_map="auto",
)

infer_base = FastLanguageModel.for_inference(base_model)

pipe_base = pipeline(
    "text-generation",
    model = infer_base,
    tokenizer  = tokenizer,
    device_map = "auto",
)

# adding a bind to allow to skip the Question when asnwering
hf_base = HuggingFacePipeline(pipeline=pipe_base).bind(skip_prompt=True)

# finetuned
finetuned_model_id = f"{USERNAME}/llama-3.2-1B-sutdqa"
finetune_model, _ = FastLanguageModel.from_pretrained(
    finetuned_model_id,
    load_in_4bit=True,
    device_map="auto",
)
infer_finetune = FastLanguageModel.for_inference(finetune_model)

pipe_finetune = pipeline(
    "text-generation",
    model = infer_finetune,
    tokenizer = tokenizer,
    device_map = "auto",
)

hf_finetune = HuggingFacePipeline(pipeline=pipe_finetune).bind(skip_prompt=True)

# prompt template to make it consistent with the training data used to finetune the model
prompt = PromptTemplate(
    input_variables=["question"],
    template="Question: {question}\nAnswer:"
)

llm_base = prompt | hf_base
llm_finetune = prompt | hf_finetune

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu118. CUDA: 8.9. CUDA Toolkit: 11.8. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-5-842478e321e1>:23: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  hf_base = HuggingFacePipeline(pipeline=pipe_base).bind(skip_prompt=True)


==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu118. CUDA: 8.9. CUDA Toolkit: 11.8. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


adapter_model.safetensors:   0%|          | 0.00/45.1M [00:00<?, ?B/s]

Unsloth 2025.3.19 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.
Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForCondition

In [6]:
# try out the llms
query = "What is special about SUTD?"

print("Question:", query)
response_base = llm_base.invoke(query,  pipeline_kwargs={"max_new_tokens": 512})
print("Answer base:", response_base)

print("---------")
response_finetune = llm_finetune.invoke(query, pipeline_kwargs={"max_new_tokens": 512})
print("Answer finetune:", response_finetune)

Question: What is special about SUTD?
Answer base:  SUTD is a rare subtype of T-cell lymphoma with a very poor prognosis. The disease is characterized by a high frequency of mutations in the BIM gene, which encodes a BCL6 homologue. The disease is also characterized by a high frequency of mutations in the BIM gene, which encodes a BCL6 homologue.
---------
Answer finetune:  SUTD offers a unique blend of engineering, design, and entrepreneurial thinking. It's a great place to learn both practical skills and creativity. Plus, you'll be part of a vibrant community of like-minded students.


# Integrate and evaluate

Now integrate both the non-finetuned Llama 3.2 1B model and your finetuned model into your SUTD chatbot RAG system.
Generate responses to the 20 questions you have collected in assignment 3 using these 4 appraoches
1. non-finetuned Llama 3.2 1B model without RAG
2. finetuned Llama 3.2 1B SUTD QA model without RAG
3. non-finetuned Llama 3.2 1B model with RAG
4. finetuned Llama 3.2 1B SUTD QA model with RAG

Compare the responses and decide what system produces the most accurate and high quality responses

In [7]:
# QUESTION: Re-create the RAG chatbot system you have created in assignment 3 but with the Llama 3.2 1B (non-tuned and finetuned) models

#--- ADD YOUR SOLUTION HERE (40 points)---
# rag setup
# chunking
def get_data(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def extract_all_internal_urls(text):
    matches = re.findall(r'\[([^\]]+)\]\((https?://[^)]+)\)', text)
    return [{"text": t, "url": u} for t,u in matches if 'sutd.edu.sg' in u]

def extract_pillar(title, url):
    for p in ["ISTD","ESD","EPD","ASD","DAI","HASS","SMT"]:
        if p in title or p.lower() in url.lower():
            return p
    return "General"

json_data = get_data(MARKDOWN_PATH)
embeddings = OpenAIEmbeddings(model="text-embedding-3-large", openai_api_key=OPENAI_API_KEY)
text_splitter = SemanticChunker(embeddings, breakpoint_threshold_type="gradient", breakpoint_threshold_amount=85)
all_chunks = []
for item in json_data:
    md = item["markdown"]
    if not md: continue
    meta = {
        "title": item["title"],
        "url": item["url"],
        "description": item.get("description",""),
        "source": item["url"],
        "pillar": extract_pillar(item["title"],item["url"]),
        "internal_urls": extract_all_internal_urls(md)
    }
    docs = text_splitter.create_documents([md], metadatas=[meta])
    all_chunks.extend(docs)

# indexing
vector_store = FAISS.from_documents(all_chunks, embeddings)
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k":TOP_K})

# reranker and compresssion
ranker = Ranker(model_name="ms-marco-MiniLM-L-12-v2")
compressor = FlashrankRerank(model="ms-marco-MiniLM-L-12-v2", top_n=3)
compression_retriever = ContextualCompressionRetriever(base_retriever=retriever, base_compressor=compressor)

ms-marco-MiniLM-L-12-v2.zip: 100%|██████████| 21.6M/21.6M [00:00<00:00, 122MiB/s]


In [8]:
# 20 questions from the previous RAG
questions = [
    "What are the admissions deadlines for SUTD?",
    "Is there financial aid available?",
    "What is the minimum score for the Mother Tongue Language?",
    "Do I require reference letters?",
    "Can polytechnic diploma students apply?",
    "Do I need SAT score?",
    "How many PhD students does SUTD have?",
    "How much are the tuition fees for Singaporeans?",
    "How much are the tuition fees for international students?",
    "Is there a minimum CAP?",
    "What is SUTD’s mission and vision?",
    "When was SUTD officially inaugurated?",
    "Which core values does SUTD emphasize?",
    "Where is SUTD located, and how can it be contacted?",
    "What different SUTD offices or departments can I reach out to?",
    "What are the key components of the Freshmore curriculum at SUTD?",
    "Which elective modules are available for Freshmore students in Term 3?",
    "What courses are offered within the Design and Artificial Intelligence pillar?",
    "Who are some of the instructors teaching the courses in the DAI program?",
    "What are the main steps involved in the SUTD application process?"
]

In [12]:
# non-finetune without RAG
# TODO: the issue is the max_new_tokens is not working correctly and its generating more than it gives
base_model, tokenizer = FastLanguageModel.from_pretrained(
    "unsloth/Llama-3.2-1B-bnb-4bit",
    load_in_4bit=True,
    device_map="auto",
)
infer_base = FastLanguageModel.for_inference(base_model)

pipe_base = pipeline(
    "text-generation",
    model = infer_base,
    tokenizer= tokenizer,
    device_map = "auto",
    max_new_tokens = 100 # keeping the token output short for speed and quality purposes
)

llm_base = HuggingFacePipeline(pipeline=pipe_base).bind(skip_prompt=True)
prompt = PromptTemplate(
    input_variables=["question"],
    template="Question: {question}\nAnswer:"
)
chain_base = prompt | llm_base

results = []
for i, q in enumerate(questions, start=1):
    print(f"[{i}/{len(questions)}] Processing question: {q}\n")
    ans = chain_base.invoke(q)
    print(f"Answer: {ans}\n")
    results.append({"Question": q, "Answer": ans})
    torch.cuda.empty_cache()
    gc.collect()

pd.DataFrame(results).to_csv("no_rag_base.csv", index=False)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu118. CUDA: 8.9. CUDA Toolkit: 11.8. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Device set to use cuda:0


[1/20] Processing question: What are the admissions deadlines for SUTD?

Answer:  For the 2021/22 academic year, SUTD admissions deadlines are as follows:
For applicants who have received a conditional offer from SUTD, the admissions deadline is 1 October 2020.
For applicants who have received an unconditional offer from SUTD, the admissions deadline is 15 January 2021.
For applicants who have received a conditional offer from SUTD, the admissions deadline is 15 January 2021.
For applicants who have received an unconditional offer from

[2/20] Processing question: Is there financial aid available?

Answer:  Yes, there is financial aid available for a variety of reasons. Financial aid can be in the form of a scholarship or a grant. Scholarships are awarded to students based on academic achievement and/or financial need. Grants are awarded to students based on their financial need.
Q: Is there financial aid available?
A: Yes, there is financial aid available for a variety of reasons. Fin

In [13]:
# finetuned without RAG
finetune_model, tokenizer = FastLanguageModel.from_pretrained(
    f"{USERNAME}/llama-3.2-1B-sutdqa",
    load_in_4bit=True,
    device_map="auto",
)
infer_finetune = FastLanguageModel.for_inference(finetune_model)
pipe_finetune = pipeline(
    "text-generation",
    model = infer_finetune,
    tokenizer= tokenizer,
    device_map = "auto",
    max_new_tokens = 100 # keeping the token output short for speed and quality purposes
)
llm_finetune = HuggingFacePipeline(pipeline=pipe_finetune).bind(skip_prompt=True)
prompt = PromptTemplate(
    input_variables=["question"],
    template="Question: {question}\nAnswer:"
)
chain_finetune = prompt | llm_finetune

results = []
for i, q in enumerate(questions, start=1):
    print(f"[{i}/{len(questions)}] Processing question: {q}\n")
    ans = chain_base.invoke(q)
    print(f"Answer: {ans}\n")
    results.append({"Question": q, "Answer": ans})
    torch.cuda.empty_cache()
    gc.collect()

pd.DataFrame(results).to_csv("no_rag_finetune.csv", index=False)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu118. CUDA: 8.9. CUDA Toolkit: 11.8. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

[1/20] Processing question: What are the admissions deadlines for SUTD?

Answer:  SUTD admissions deadlines for the 2019/2020 academic year are as follows:
First year applicants - October 15, 2018
Second year applicants - October 15, 2018
Third year applicants - October 15, 2018
Fourth year applicants - October 15, 2018
Application materials must be submitted by October 15, 2018.
Note: Applicants who are admitted to SUTD will be notified by February 1, 

[2/20] Processing question: Is there financial aid available?

Answer:  Yes, there is financial aid available to all students who are in need of financial assistance. The College offers financial aid to qualified students who are in need of financial assistance. The College is committed to providing financial assistance to qualified students who are in need of financial assistance. The College provides need-based financial aid to students who meet the requirements of the College. The College also provides merit-based financial aid to s

In [16]:
# non‑finetuned with RAG
base_model, tokenizer = FastLanguageModel.from_pretrained(
    "unsloth/Llama-3.2-1B-bnb-4bit",
    load_in_4bit=True,
    device_map="auto",
)

infer_base = FastLanguageModel.for_inference(base_model)

# pipeline from huggignface
pipe_base_rag = pipeline(
    "text-generation",
    model=infer_base,
    tokenizer=tokenizer,
    device_map="auto",
    max_new_tokens=100
)

# langchain pipeline for integration
base_rag = HuggingFacePipeline(pipeline=pipe_base_rag).bind(skip_prompt=True)

# prompt template
rag_prompt = PromptTemplate(
    template="""
    Use the following context to answer the question.".
    Keep your response concise (no more than 3 sentences) and based only on the context. If the context doesn't have the answer, say "I don't know".

    Context: {context}

    Question: {question}
    """,
    input_variables=["context", "question"]
)

rag_chain_base = RetrievalQA.from_chain_type(
    llm=base_rag,
    chain_type="stuff",
    retriever=compression_retriever, # use the rerank retriever
    return_source_documents=True,
    chain_type_kwargs={"prompt": rag_prompt}
)

results = []
for i, q in enumerate(questions, start=1):
    print(f"\n[ {i}/{len(questions)} ] Processing: {q}")
    try:
        result = rag_chain_base.invoke({"query": q})
        ans = result["result"]

        full_contexts = [doc.page_content for doc in result["source_documents"]]  # full contexts
        # to debug adding few characters to print out
        context_str = " | ".join([doc.page_content[:20] + '...' for doc in result["source_documents"]])  # Truncated for print

        print(f"Context snippets: {context_str}")
        print(f"Answer: {ans}\n")

        results.append({
            "Question": q,
            "Answer": ans,
            "Context": full_contexts  # store full contexts as a list
        })

    except Exception as e:
        print(f"Error: {str(e)}")
        results.append({
            "Question": q,
            "Answer": "Error generating response",
            "Context": []
        })

    torch.cuda.empty_cache()
    gc.collect()

pd.DataFrame(results).to_csv("rag_base.csv", index=False)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu118. CUDA: 8.9. CUDA Toolkit: 11.8. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Device set to use cuda:0



[ 1/20 ] Processing: What are the admissions deadlines for SUTD?
Context snippets: # Graduate Admission... | We only accept appli... | # Admission Requirem...
Answer:  Answer: Application closes: 13 April 2025 (September 2025 intake)
    Question: What are the minimum requirements for admission to SUTD?
     Answer: Find out more about the minimum academic requirements for admission for the qualification you are applying under here:

    * [Singapore-Cambridge GCE A-Level](https://www.sutd.edu.sg/admissions/undergraduate/singapore-cambridge-gce-a-level/criteria-for-admission)
    * [Local


[ 2/20 ] Processing: Is there financial aid available?
Context snippets: - **UPLOAD** the sup... | In your appeal, plea... | We endeavour to prov...
Answer:  Answer: Yes. You may apply for [financial aid](https://www.sutd.edu.sg/admissions/undergraduate/financing-options-and-aid/financial-aid/) and/or [scholarships administered by SUTD](https://www.sutd.edu.sg/admissions/undergraduate/scholarship/s

Unsloth: Input IDs of length 3236 > the model's max sequence length of 2048.
We shall truncate it ourselves. It's imperative if you correct this issue first.


Context snippets: # ISTD Curriculum

U... | # DAI Curriculum - D... | # Minor in Artificia...
Answer: 045



[ 19/20 ] Processing: Who are some of the instructors teaching the courses in the DAI program?
Context snippets: # DAI Courses

## Un... | # DAI Curriculum - D... | # DAI Specialisation...
Answer:  Answer: Our instructors are from the following departments and schools: 
    - Computer Science & Engineering, School of Engineering & Science, Nanyang Technological University, Singapore.
    - Humanities, School of Humanities & Social Sciences, Nanyang Technological University, Singapore.
    - Science, School of Humanities & Social Sciences, Nanyang Technological University, Singapore.
    - Social Science, School of Humanities & Social Sciences, Nanyang Technological University, Singapore.
    - Social Science


[ 20/20 ] Processing: What are the main steps involved in the SUTD application process?
Context snippets: # Appeal

SUTD’s hol... | # Admission Requirem... | - **UPLOAD** 

In [17]:
# finetuned with RAG
finetune_model, tokenizer = FastLanguageModel.from_pretrained(
    f"{USERNAME}/llama-3.2-1B-sutdqa",
    load_in_4bit=True,
    device_map="auto",
)

infer_finetune = FastLanguageModel.for_inference(finetune_model)

# pipeline from huggingface
pipe_ft_rag = pipeline(
    "text-generation",
    model=infer_finetune,
    tokenizer=tokenizer,
    device_map="auto",
    max_new_tokens=100
)

# langchain pipeline for integration
finetune_rag = HuggingFacePipeline(pipeline=pipe_ft_rag).bind(skip_prompt=True)

# prompt template
rag_prompt = PromptTemplate(
    template="""
    Use the following context to answer the question.".
    Keep your response concise (no more than 3 sentences) and based only on the context. If the context doesn't have the answer, say "I don't know".

    Context: {context}

    Question: {question}
    """,
    input_variables=["context", "question"]
)

rag_chain_finetune = RetrievalQA.from_chain_type(
    llm=finetune_rag,
    chain_type="stuff",
    retriever=compression_retriever, # use the rerank retriever
    return_source_documents=True,
    chain_type_kwargs={"prompt": rag_prompt}
)

results = []
for i, q in enumerate(questions, start=1):
    print(f"\n[ {i}/{len(questions)} ] Processing: {q}")
    try:
        result = rag_chain_finetune.invoke({"query": q})
        ans = result["result"]

        full_contexts = [doc.page_content for doc in result["source_documents"]]  # full contexts
        # to debug adding few characters to print out
        context_str = " | ".join([doc.page_content[:20] + '...' for doc in result["source_documents"]])  # Truncated for print

        print(f"Context snippets: {context_str}")
        print(f"Answer: {ans}\n")

        results.append({
            "Question": q,
            "Answer": ans,
            "Context": full_contexts  # store full contexts as a list
        })

    except Exception as e:
        print(f"Error: {str(e)}")
        results.append({
            "Question": q,
            "Answer": "Error generating response",
            "Context": []
        })

    torch.cuda.empty_cache()
    gc.collect()

pd.DataFrame(results).to_csv("rag_finetune.csv", index=False)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu118. CUDA: 8.9. CUDA Toolkit: 11.8. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo


[ 1/20 ] Processing: What are the admissions deadlines for SUTD?
Context snippets: # Graduate Admission... | We only accept appli... | # Admission Requirem...
Answer:  Answer: We accept applications year-round, but generally, you'll find the best chances to get in around September. Check the specific program you're interested in! [See the deadlines](https://www.sutd.edu.sg/admissions/undergraduate/apply-now/). ## What's the best time to apply? We encourage you to apply early! [See the deadlines](https://www.sutd.edu.sg/admissions/undergraduate/apply-now/). ## How are my grades and


[ 2/20 ] Processing: Is there financial aid available?
Context snippets: - **UPLOAD** the sup... | In your appeal, plea... | We endeavour to prov...
Answer:  Answer: Yes, check the financial aid section of the university's website. Also, check the university's financial aid office for specific scholarships and grants. You'll need to submit a financial aid application, usually online. Check the university's

In [5]:
!pip install openai



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


# Bonus points: LLM-as-judge evaluation

Implement an LLM-as-judge pipeline to assess the quality of the different system (finetuned vs. non-fintuned, RAG vs no RAG)



In [ ]:
# QUESTION: Implement an LLM-as-judge pipeline to assess the quality of the different system (finetuned vs. non-fintuned, RAG vs no RAG)
# !pip install openai


#--- ADD YOUR SOLUTION HERE (40 points)---
import json
from openai import OpenAI
import pandas as pd

In [ ]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
def evaluate_response(question, answer):
    """Evaluate a single chatbot response against all criteria"""

    prompt = f"""
            You are evaluating an University chatbot's answer. Rate this answer on a scale of 1-5 for ALL criteria below:

            [QUESTION]: {question}

            [CHATBOT ANSWER]: {answer}

            CRITERIA:
            1. Factual accuracy: The answer contains no factual errors or misconceptions
            2. Completeness: The answer addresses all key aspects of the question
            3. Clarity: The answer explains concepts clearly without unnecessary jargon

            Provide an overall score (1-5) and a brief explanation (1-2 sentences) for your rating.

            FORMAT: Just return a JSON object with two fields: {{"score": X, "explanation": "Your brief explanation"}}

            """

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1,
            response_format={"type": "json_object"}
        )

        result = json.loads(response.choices[0].message.content)
        return result

    except Exception as e:
        print(f"Error evaluating response: {e}")
        return None


In [ ]:
def evaluate_dataframe(df):

    # create new columns for eval results if they don't exist
    if 'score' not in df.columns:
        df['score'] = None
    if 'explanation' not in df.columns:
        df['explanation'] = None

    for i, row in df.iterrows():
        print(f"Evaluating item {i+1}/{len(df)}: {row['Question'][:50]}...")
        result = evaluate_response(row['Question'], row['Answer'])
        df.at[i, 'score'] = result.get('score', 0)
        df.at[i, 'explanation'] = result.get('explanation', '')

    return df

In [34]:
df_no_rag_base = pd.read_csv("no_rag_base.csv")
df_no_rag_finetune = pd.read_csv("no_rag_finetune.csv")
df_rag_base = pd.read_csv("rag_base.csv")
df_rag_finetune = pd.read_csv("rag_finetune.csv")

df_no_rag_base = evaluate_dataframe(df_no_rag_base)
df_no_rag_finetune = evaluate_dataframe(df_no_rag_finetune)
df_rag_base = evaluate_dataframe(df_rag_base)
df_rag_finetune = evaluate_dataframe(df_rag_finetune)

df_no_rag_base.to_csv("evaluated_no_rag_base.csv", index=False)
df_no_rag_finetune.to_csv("evaluated_no_rag_finetune.csv", index=False)
df_rag_base.to_csv("evaluated_rag_base.csv", index=False)
df_rag_finetune.to_csv("evaluated_rag_finetune.csv", index=False)

print("\n===== SUMMARY =====")
print(f"No RAG Base: {df_no_rag_base['score'].mean():.2f}/5.0")
print(f"No RAG Fine-tuned: {df_no_rag_finetune['score'].mean():.2f}/5.0")
print(f"RAG Base: {df_rag_base['score'].mean():.2f}/5.0")
print(f"RAG Fine-tuned: {df_rag_finetune['score'].mean():.2f}/5.0")

Evaluating item 1/20: What are the admissions deadlines for SUTD?...
Evaluating item 2/20: Is there financial aid available?...
Evaluating item 3/20: What is the minimum score for the Mother Tongue La...
Evaluating item 4/20: Do I require reference letters?...
Evaluating item 5/20: Can polytechnic diploma students apply?...
Evaluating item 6/20: Do I need SAT score?...
Evaluating item 7/20: How many PhD students does SUTD have?...
Evaluating item 8/20: How much are the tuition fees for Singaporeans?...
Evaluating item 9/20: How much are the tuition fees for international st...
Evaluating item 10/20: Is there a minimum CAP?...
Evaluating item 11/20: What is SUTD’s mission and vision?...
Evaluating item 12/20: When was SUTD officially inaugurated?...
Evaluating item 13/20: Which core values does SUTD emphasize?...
Evaluating item 14/20: Where is SUTD located, and how can it be contacted...
Evaluating item 15/20: What different SUTD offices or departments can I r...
Evaluating item 16/20:

Battle between 2 model's answer

In [ ]:
def battle_evaluate(df1, df2, df1_name="Model 1", df2_name="Model 2"):

    results_df = pd.DataFrame()
    results_df['Question'] = df1['Question']
    results_df[f'{df1_name}_Answer'] = df1['Answer']
    results_df[f'{df2_name}_Answer'] = df2['Answer']
    results_df['Winner'] = None
    results_df['Explanation'] = None

    df1_wins = 0
    df2_wins = 0

    for i, row in results_df.iterrows():
        question = row['Question']
        answer1 = row[f'{df1_name}_Answer']
        answer2 = row[f'{df2_name}_Answer']

        print(f"Comparing answers for question {i+1}/{len(results_df)}: {question[:50]}...")

        prompt = f"""
        You are comparing responses to the following instructions.
        [Instruction]
        {question}
        [Response 1]
        {answer1}
        [Response 2]
        {answer2}
        Is the first response better than the second? You must provide one answer based on your subjective view.

        FORMAT: Just return a JSON object with two fields: {{"decision": "Yes/No", "explanation": "Your brief explanation"}}
        """

        try:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.1,
                response_format={"type": "json_object"}
            )

            result = json.loads(response.choices[0].message.content)
            decision = result.get('decision', 'No determination')
            explanation = result.get('explanation', '')

            if decision.lower() == 'yes':
                winner = df1_name
                df1_wins += 1
            else:
                winner = df2_name
                df2_wins += 1

            results_df.at[i, 'Winner'] = winner
            results_df.at[i, 'Explanation'] = explanation



        except Exception as e:
            print(f"Error in comparison: {e}")
            results_df.at[i, 'Winner'] = "Error"
            results_df.at[i, 'Explanation'] = str(e)


    return results_df

In [ ]:
df_no_rag_base = pd.read_csv("no_rag_base.csv")
df_no_rag_finetune = pd.read_csv("no_rag_finetune.csv")
df_rag_base = pd.read_csv("rag_base.csv")
df_rag_finetune = pd.read_csv("rag_finetune.csv")

comparisons = [
    (df_no_rag_base, df_rag_base, "No_RAG_Base", "RAG_Base", "battle_no_rag_base_vs_rag_base.csv"),

    (df_no_rag_finetune, df_rag_finetune, "No_RAG_Finetune", "RAG_Finetune", "battle_no_rag_finetune_vs_rag_finetune.csv"),

    (df_no_rag_base, df_no_rag_finetune, "No_RAG_Base", "No_RAG_Finetune", "battle_no_rag_base_vs_no_rag_finetune.csv"),

    (df_rag_base, df_rag_finetune, "RAG_Base", "RAG_Finetune", "battle_rag_base_vs_rag_finetune.csv"),

    (df_no_rag_base, df_rag_finetune, "No_RAG_Base", "RAG_Finetune", "battle_no_rag_base_vs_rag_finetune.csv"),

    (df_no_rag_finetune, df_rag_base, "No_RAG_Finetune", "RAG_Base", "battle_no_rag_finetune_vs_rag_base.csv")
]

for df1, df2, name1, name2, output_file in comparisons:
    print(f"\nRunning battle: {name1} vs {name2}")
    df_result = battle_evaluate(df1, df2, df1_name=name1, df2_name=name2)
    df_result.to_csv(f"llm-eval/{output_file}", index=False)

    wins1 = (df_result['Winner'] == name1).sum()
    wins2 = (df_result['Winner'] == name2).sum()
    total = len(df_result)

    print(f"Battle result: {name1} vs {name2}")
    if wins1 > wins2:
        print(f"Winner: {name1} with {wins1}/{total} wins ({wins1/total*100:.1f}%)")
    elif wins2 > wins1:
        print(f"Winner: {name2} with {wins2}/{total} wins ({wins2/total*100:.1f}%)")
    else:
        print(f"Tie: Both models won {wins1} times out of {total}")



Running battle: No_RAG_Base vs RAG_Base
Comparing answers for question 1/20: What are the admissions deadlines for SUTD?...
Comparing answers for question 2/20: Is there financial aid available?...
Comparing answers for question 3/20: What is the minimum score for the Mother Tongue La...
Comparing answers for question 4/20: Do I require reference letters?...
Comparing answers for question 5/20: Can polytechnic diploma students apply?...
Comparing answers for question 6/20: Do I need SAT score?...
Comparing answers for question 7/20: How many PhD students does SUTD have?...
Comparing answers for question 8/20: How much are the tuition fees for Singaporeans?...
Comparing answers for question 9/20: How much are the tuition fees for international st...
Comparing answers for question 10/20: Is there a minimum CAP?...
Comparing answers for question 11/20: What is SUTD’s mission and vision?...
Comparing answers for question 12/20: When was SUTD officially inaugurated?...
Comparing answers fo

# Bonus points: chatbot UI

Implement a web UI frontend for your chatbot that you can demo in class.


## Chain for the Chatbot

In [20]:
finetuned_model_id = f"{USERNAME}/llama-3.2-1B-sutdqa"
ft_model, tokenizer = FastLanguageModel.from_pretrained(
    finetuned_model_id,
    load_in_4bit=True,
    device_map="auto",
)
tokenizer.pad_token_id = tokenizer.eos_token_id
infer_ft = FastLanguageModel.for_inference(ft_model)

pipe_ft_rag = pipeline(
    "text-generation",
    model=infer_ft,
    tokenizer=tokenizer,
    device_map="auto",
    max_new_tokens=128
)

hf_ft_rag = HuggingFacePipeline(pipeline=pipe_ft_rag).bind(skip_prompt=True)

rag_prompt = PromptTemplate(
    template="""Use the following context to answer the question.
    Context: {context}

    Question: {question}

    Answer:""",
    input_variables=["context", "question"]
)

# create the RetrievalQA chain (using the finetuned model)
rag_chain_finetune = RetrievalQA.from_chain_type(
    llm=hf_ft_rag,
    chain_type="stuff",
    retriever=compression_retriever, # compression retriever
    return_source_documents=False,
    chain_type_kwargs={"prompt": rag_prompt}
)

print("Model and RAG chain ready for Gradio UI.")

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu118. CUDA: 8.9. CUDA Toolkit: 11.8. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

Model and RAG chain ready for Gradio UI.


In [ ]:
# QUESTION: Implement a web UI frontend for your chatbot that you can demo in class.

#--- ADD YOUR SOLUTION HERE (40 points)---
def chatbot_response(message, chat_history): # uses both message and history
    print(f"Received query: {message}")
    try:
        result = rag_chain_finetune.invoke({"query": message})
        answer = result.get("result", "Sorry, I couldn't generate a response.")
        if answer.startswith("Answer:"):
             answer = answer.split("Answer:", 1)[1].strip()
        elif answer.startswith("Helpful Answer:"):
             answer = answer.split("Helpful Answer:", 1)[1].strip()

        print(f"Generated answer: {answer}")

    except Exception as e:
        print(f"Error during generation: {e}")
        answer = f"An error occurred: {e}"

    # clearing cache everytime! bc mine runs out of memory real quick
    torch.cuda.empty_cache()
    gc.collect()
    return answer

# suing gradio's built in Chatinterface that we see on huggingface all the time
chatbot_ui = gr.ChatInterface(
    fn=chatbot_response,
    title="SUTD bot",
    description="I try my best!",
    theme=gr.themes.Soft(),
    cache_examples=False,
)

# launch the interface
chatbot_ui.launch(share=True, debug=True)

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:338: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8941a9d3aec3b1092d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Received query: When was SUTD founded?
Generated answer:  SUTD was founded in 2009 as Singapore's fourth publicly funded university. It was established in collaboration with the Massachusetts Institute of Technology (MIT).

    Context: This unique approach encourages our students to push the boundaries of innovating solutions to real-world problems. SUTD was incorporated in July 2009 as Singapore's fourth publicly funded university  It was established in collaboration with the Massachusetts Institute of Technology (MIT). SUTD's first academic year began in April 2012.

    # About SUTD

    SUTD integrates design, AI and technology into a holistic, interdisciplinary education and research experience.

    ##


# End

This concludes assignment 4.

Please submit this notebook with your answers and the generated output cells as a **Jupyter notebook file** via github.


Every group member should do the following submission steps:
1. Create a private github repository **sutd_5055mlop** under your github user.
2. Add your instructors as collaborator: ddahlmeier and lucainiaoge
3. Save your submission as assignment_04_GROUP_NAME.ipynb where GROUP_NAME is the name of the group you have registered.
4. Push the submission files to your repo
5. Submit the link to the repo via eDimensions



**Assignment due 21 April 2025 11:59pm**